# Summary

**Action Points**

**1) Prioritize retention and risk mitigation in the "Online TA" segment**<br>
- Accounts for >50% of all successful bookings
- Has been more volatile in 2020 YTD versus equivalent dates 2019:
    - 32% rise in cancellations, therefore responsible for 106% of increase in cancellations
- Stabilising this segment is critical to maintaining overall booking volume and revenue consistency

**2) Reassess strategy for the "Offline TA/TO" segment**<br>
- Improvement in cancellations (-20%) in 2020 YTD versus equivalent dates 2019
- However, number of actual revenue-generating bookings dropped 36% - this is the biggest contributor to the 2020 YTD revenue shortfall, as this segment makes up about 23% of successful bookings (completed, or cancelled and non-refundable)
- Revival is a high priority
- Ideas examples:
    - Increased engagement with travel agents
    - Review of channel-specific pricing and package offerings

**3) Investigate root causes of longer lead times in general**<br>
- Whilst lead times for completed bookings have increased, so too have lead times for cancelled bookings, in 2020 YTD versus equivalent dates 2019.
- I.e. indicator of potential fragility: disturbs confidence in long-term revenue forecasting and management
- Possibility: previously introduced feature unintentially made reservations more tentative

**Fly-by:**

1) Revenue for 2018 (partial) far surpassed by equivalent-period 2019
- Main driver is number of bookings, not increased value of bookings
    - Especially in market segment **"Online TA"**, but also worth highlighting **"Offline TA/TO"** and **"Direct"**

2) YTD revenue 2020 almost matches that of equivalent-period 2019
- Sustainable: consistent month-by-month revenue growth in 2020
- Same three important market segments as mentioned above contributed proportionally to overall number of bookings as much in both 2019 and 2020
- However, number of revenue-generating bookings by secondary key segment "Offline TA/TO" decreased 36% in 2020 YTD vs 2019 YTD - _action point 2_

3) Encouraging: though number of bookings overall decreased, quality went up ("quality booking" := generated revenue)

4) Encouraging: longer average lead times in 2020 vs 2019, both overall and specifically in reference to those 3 major segments
- However,  lead time of cancelled bookings increased too, again for all 3 major segments - _action point 3_

5) (4) is especially noteworthy given the completion rate of bookings fell and actual number of cancellations rose in general, and most apparently in the key segment "Online TA" - _action point 1_

# Preamble

Dataset outline: 2018 - 2020 bookings for Hotel portfolio<br>
https://absentdata.com/data-analysis/where-to-find-data/<br>

Additional info on definitions:<br>
https://github.com/rfordatascience/tidytuesday/blob/main/data/2020/2020-02-11/readme.md
- Concat this data in future revisions

High-level findings and recommendations:


Summary of concerns and assumptions:
1) Assume SC meals (self-catered) have not had their price written in error, despite somehow costing more than full-board
2) Assume 'adr' (avg daily rate) is to be multiplied by nights stayed to find overall room rental revenue per booking, exclusive of meal costs
- See source: "Average Daily Rate as defined by dividing the sum of all lodging transactions by the total number of staying nights"
3) Assume meals are applied to every non-baby guest in the same booking, for every day (nights + 1) spent
4) Assume Null children --> 0 children
5) Assume no money made from cancelled bookings. Or rather, do admit possibility of reclaiming deposits, but assess revenues of strictly completed bookings first
6) Assume bookings of zero pax are unsuable data
7) Assume bookings with zero adr (avg daily rate) yet which are not cancelled are unusable data

# Prelim

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
# Remove scientific notation in larger figures
pd.options.display.float_format = '{:.2f}'.format

## Load Data

In [4]:
# 2018 to 2020
# sheet_name = None --> all worksheets
df1 = pd.read_excel(r"D:\Code\Data Analysis\01 Hotel Revenue\hotel_revenue_historical_full-2.xlsx", sheet_name = None)

In [5]:
# 2015 to 2017
df2 = pd.read_csv(r"D:\Code\Data Analysis\01 Hotel Revenue\hotels.csv")

In [6]:
# Concat sheets within df1, having verified in source that columns match. Also from df2.
# ---> Consolidated fact table of bookings data 2015 to 2020
df_main = pd.concat([df1['2018'], df1['2019'], df1['2020'], df2])
pd.set_option("display.max.columns", 50)

In [7]:
df_main[df_main["is_canceled"] == 0].head(10)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
7951,Resort Hotel,0,342,2018,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2018-07-01 00:00:00
7952,Resort Hotel,0,737,2018,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2018-07-01 00:00:00
7953,Resort Hotel,0,7,2018,July,27,1,0,1,1,0.00,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2018-07-02 00:00:00
7954,Resort Hotel,0,13,2018,July,27,1,0,1,1,0.00,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.00,NaN,0,Transient,75.00,0,0,Check-Out,2018-07-02 00:00:00
7955,Resort Hotel,0,14,2018,July,27,1,0,2,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.00,NaN,0,Transient,98.00,0,1,Check-Out,2018-07-03 00:00:00
7956,Resort Hotel,0,14,2018,July,27,1,0,2,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.00,NaN,0,Transient,98.00,0,1,Check-Out,2018-07-03 00:00:00
7957,Resort Hotel,0,0,2018,July,27,1,0,2,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,NaN,NaN,0,Transient,107.00,0,0,Check-Out,2018-07-03 00:00:00
7958,Resort Hotel,0,9,2018,July,27,1,0,2,2,0.00,0,FB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,303.00,NaN,0,Transient,103.00,0,1,Check-Out,2018-07-03 00:00:00
7959,Resort Hotel,0,35,2018,July,27,1,0,4,2,0.00,0,HB,PRT,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,240.00,NaN,0,Transient,145.00,0,0,Check-Out,2018-07-05 00:00:00
7960,Resort Hotel,0,68,2018,July,27,1,0,4,2,0.00,0,BB,USA,Online TA,TA/TO,0,0,0,D,E,0,No Deposit,240.00,NaN,0,Transient,97.00,0,3,Check-Out,2018-07-05 00:00:00


In [8]:
# Dimension table 1
# Typically would assume SC (self-catered) --> cheaper
# Seems not the case this time. Raise concern of error, but go along with it (do not drop SC or any such action)

df_meal_cost = df1['meal_cost']
df_meal_cost

,Cost,meal
0,0.00,Undefined
1,12.99,BB
2,17.99,HB
3,21.99,FB
4,35.00,SC


In [9]:
# Dimension table 2
# Note discounted price would be price * (1 - discount)

df_market_segment = df1['market_segment']
df_market_segment

,Discount,market_segment
0,0.00,Undefined
1,0.10,Direct
2,0.10,Groups
3,0.15,Corporate
4,0.20,Aviation
5,0.30,Offline TA/TO
6,0.30,Online TA
7,1.00,Complementary


# Clean and Structure Data

## Dupes, Nulls, Anomalies

In [10]:
# Any dupes?
# 73185
len(df_main[df_main.duplicated()])

73185

In [11]:
# Can't find justification for why these dupes need to be kept, so drop them
df_main.drop_duplicates(inplace=True)
len(df_main[df_main.duplicated()])

0

In [12]:
df_main.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                              12
babies                                 0
meal                                   0
country                             1013
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              26732
company         

Note: null categorical data like 'agent' and 'company' explained in data dictionary as meaning N/A, not missing in value

In [13]:
# Assumption: null children = 0. Need non-null to convert to int (from float), in next cell
df_main["children"] = df_main["children"].fillna(value = 0)

In [14]:
# Convert datatypes:
    # children to integer from float
    # reservation_status_date to datetime from object
df_main["children"] = df_main["children"].astype(np.int64)
df_main["reservation_status_date"] = pd.to_datetime(df_main["reservation_status_date"])#.dt.date
df_main.dtypes

hotel                                     object
is_canceled                                int64
lead_time                                  int64
arrival_date_year                          int64
arrival_date_month                        object
arrival_date_week_number                   int64
arrival_date_day_of_month                  int64
stays_in_weekend_nights                    int64
stays_in_week_nights                       int64
adults                                     int64
children                                   int64
babies                                     int64
meal                                      object
country                                   object
market_segment                            object
distribution_channel                      object
is_repeated_guest                          int64
previous_cancellations                     int64
previous_bookings_not_canceled             int64
reserved_room_type                        object
assigned_room_type  

## Feature Engineering, Further Cleaning

In [15]:
# Convert month from English name ('July') to number (7)
df_main["arrival_date_month"] = pd.to_datetime(df_main["arrival_date_month"], format = "%B").dt.month

# Combine separate arrival date columns into singular date column
df_main["arrival_date"] = pd.to_datetime({
    'year': df_main['arrival_date_year'],
    'month': df_main['arrival_date_month'],
    'day': df_main['arrival_date_day_of_month']
})

In [16]:
df_main[["arrival_date", "arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"]].head(15)

,arrival_date,arrival_date_year,arrival_date_month,arrival_date_day_of_month
0,2018-07-01,2018,7,1
1,2018-07-01,2018,7,1
2,2018-07-01,2018,7,1
3,2018-07-01,2018,7,1
4,2018-07-01,2018,7,1
5,2018-07-02,2018,7,2
6,2018-07-02,2018,7,2
7,2018-07-02,2018,7,2
8,2018-07-02,2018,7,2
9,2018-07-02,2018,7,2


In [17]:
# Don't drop - easier filtering if retained
#df_main = df_main.drop(labels = ["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"], axis = 1)

In [18]:
df_main["pax"] = df_main["adults"] + df_main["children"] + df_main["babies"]

In [19]:
len(df_main[df_main["pax"] == 0])

361

In [20]:
# Absent deeper knowledge on booking system, assume 0 pax bookings are in error / not usable
df_main = df_main[df_main["pax"] != 0].copy()

In [21]:
# adr := sum of all lodging transactions divided by total number of staying nights
# So sum of all lodging transactions per booking (row) is adr * nights

df_main["nights"] = df_main["stays_in_week_nights"] + df_main["stays_in_weekend_nights"]
df_main[["nights", "stays_in_week_nights", "stays_in_weekend_nights"]].head(10)

,nights,stays_in_week_nights,stays_in_weekend_nights
0,3,3,0
1,3,3,0
2,4,4,0
3,7,5,2
4,10,8,2
5,4,3,1
6,4,3,1
7,4,3,1
8,5,3,2
9,7,5,2


In [22]:
# 3,634 entries are not cancelled yet have adr (avg daily rate) of 0.
# adr is "calculated by dividing the sum of all lodging transactions by the total number of staying nights"
df_main.groupby([(df_main["adr"] == 0) & (df_main["is_canceled"] == 0)]).size()

False    184422
True       3369
dtype: int64

In [23]:
# There are examples below of adr = 0, canceled = 0, and nights != 0
# Even if strictly nights = 0 for these conditions, that wouldn't easily explain anything, but might have been a hint into reasoning

# df_main[["nights", "stays_in_weekend_nights", "stays_in_week_nights"]][(df_main["adr"] == 0) & (df_main["is_canceled"] == 0)].sort_values(by = df_main["nights"], ascending=False)

df_main[(df_main["adr"] == 0) & (df_main["is_canceled"] == 0)][["nights", "stays_in_weekend_nights", "stays_in_week_nights"]].sort_values(by="nights", ascending=False)

,nights,stays_in_weekend_nights,stays_in_week_nights
34715,48,14,34
54704,48,14,34
1655,46,13,33
8989,46,13,33
65696,46,13,33
...,...,...,...
115483,0,0,0
117701,0,0,0
118029,0,0,0
18836,0,0,0


In [24]:
# Examples below show that cases of adr = 0 & is_canceled = 0 do not congregate around one particular dimension value,
# which would hint that the result is systematic --> stronger evidence that this is erroneous / not by design
df_main[(df_main["adr"] == 0) & (df_main["is_canceled"] == 0)].groupby("market_segment").size().sort_values(ascending = False)
# df_main[(df_main["adr"] == 0) & (df_main["is_canceled"] == 0)].groupby("distribution_channel").size().sort_values(ascending = False)
# df_main[(df_main["adr"] == 0) & (df_main["is_canceled"] == 0)].groupby("customer_type").size().sort_values(ascending = False)

market_segment
Complementary    1228
Online TA         661
Offline TA/TO     487
Groups            451
Direct            404
Corporate         132
Aviation            6
dtype: int64

In [25]:
# Note too that "Complementary" market segment discount of 100% is applied in post, as seen by max 'adr' for this segment being > 0
df_main.groupby(df_main["market_segment"] == "Complementary")["adr"].max()

market_segment
False   5400.00
True     170.00
Name: adr, dtype: float64

In [26]:

# Absent more detailed knowledge of subject matter, assume the examples found are unusable
# Even if these are explained by e.g. vouchers and perks, there is no alternate column to demonstrate this anyway
df_main = df_main[~((df_main["adr"] == 0) & (df_main["is_canceled"] == 0))].copy()
# After slicing, e.g. as above, Pandas does not necessarily create a totally new df_main, rather just a shallow copy, or even just a view
# So later when e.g. adding new columns, Pandas not sure if applying to original or view, and changes may or may not exist
# Therefore make a deep copy

In [27]:
# reservation_status := ['canceled', 'check out', 'no show']
# reservation_status_date := "Date at which the last status was set [...] understand when was the booking canceled or when did the customer checked-out of the hotel"
df_main.groupby("arrival_date_year")["reservation_status_date"].agg(['min', 'max'])
# (Later standardise the format of column)

,min,max
arrival_date_year,,
2015,2014-10-17,2016-01-14
2016,2015-06-26,2017-01-14
2017,2015-10-21,2017-09-14
2018,2014-10-17,2019-01-14
2019,2014-10-17,2020-01-14
2020,2018-10-21,2020-09-14


In [28]:
# Some min dates look suspicious
# Technically, would be possible in the case of cancellations for the reserve_status_date to preceded arrival_date

# First see if any cases where reserve_status_date < arrival_date but corresponding status was not cancellation
# Where 'is_canceled' = 1 yet reservation_status is not 'Canceled', we prioritise the reservation_status, to align with documentation

# df_main[(df_main["reservation_status_date"] < df_main["arrival_date"]) & (df_main["reservation_status"] != 'Canceled')]
df_main[["is_canceled", "reservation_status", "reservation_status_date", "arrival_date"]][(df_main["reservation_status_date"] < df_main["arrival_date"]) & (df_main["reservation_status"] != 'Canceled')]

,is_canceled,reservation_status,reservation_status_date,arrival_date
56104,1,No-Show,2019-03-28,2019-07-17
66832,0,Check-Out,2019-01-05,2019-12-08
66852,0,Check-Out,2019-01-05,2019-12-17
66889,0,Check-Out,2019-01-02,2019-12-23
66905,0,Check-Out,2019-01-01,2019-12-24
...,...,...,...,...
79259,1,Check-Out,2019-01-06,2019-03-01
79260,1,Check-Out,2018-11-20,2019-03-02
79261,1,Check-Out,2018-11-20,2019-03-02
79263,1,Check-Out,2019-01-27,2019-03-02


In [29]:
# 328 rows cannot be explained - delete
# Exclude the 328 problematic rows
df_main = df_main[~((df_main["reservation_status_date"] < df_main["arrival_date"]) & (df_main["reservation_status"] != 'Canceled'))].copy()

In [30]:
# lead_time := number of days [...]] between the entering date of the booking into the PMS and the arrival date
# PMS := Property Management System

# Wouldn't make sense for reservation_status_date to precede the booking_date in any scenario (cancellation, no show, check out)
# Derive booking_date from datediff(arrival - lead time) then compare:


df_main["booking_date"] = df_main["arrival_date"] - pd.to_timedelta(df_main["lead_time"], unit='days')
df_main[["is_canceled", "reservation_status", "reservation_status_date", "arrival_date", "lead_time", "booking_date"]][df_main["booking_date"] > df_main["reservation_status_date"]]

,is_canceled,reservation_status,reservation_status_date,arrival_date,lead_time,booking_date
566,1,Canceled,2014-11-18,2018-09-03,297,2017-11-10
4215,1,Canceled,2014-10-17,2018-07-09,265,2017-10-17
4216,1,Canceled,2014-10-17,2018-07-02,258,2017-10-17
4236,1,Canceled,2014-10-17,2018-07-09,265,2017-10-17
4255,1,Canceled,2014-10-17,2018-07-16,272,2017-10-17
...,...,...,...,...,...,...
15287,1,Canceled,2020-01-09,2020-08-25,228,2020-01-10
15325,1,Canceled,2020-02-12,2020-08-26,195,2020-02-13
15346,1,Canceled,2020-02-19,2020-08-27,189,2020-02-20
15413,1,Canceled,2020-02-04,2020-08-30,207,2020-02-05


In [31]:
# Can't explain above 688 rows where booking_date > reservation_status_date ("latest" status), so delete:

df_main = df_main[~(df_main["booking_date"] > df_main["reservation_status_date"])].copy()

In [32]:
# reservation_status == 'Canceled' := customer cancelled
# is_canceled == 1 := cancelled on booking system ... presumably by hotel operator
# So below instances of reservation_status == 'Check-Out' but is_canceled == 1 don't make sense?

df_main[(df_main["is_canceled"] == 1)].groupby("reservation_status").size()

reservation_status
Canceled     47852
Check-Out        2
No-Show       2179
dtype: int64

In [33]:
# Drop 2 instances of is_canceled == 1 and reservation_status == 'Check-Out'
df_main = df_main[~((df_main["is_canceled"] == 1) & (df_main["reservation_status"] == 'Check-Out'))].copy()

In [34]:
df_main[(df_main["is_canceled"] == 0)].groupby("reservation_status").size()

reservation_status
Check-Out    133373
dtype: int64

### Seasonal Analysis Set-up

In [35]:
df_main.groupby("arrival_date_year")["arrival_date"].agg(['min', 'max']).reset_index()

,arrival_date_year,min,max
0,2015,2015-07-01,2015-12-31
1,2016,2016-01-01,2016-12-31
2,2017,2017-01-01,2017-08-31
3,2018,2018-07-01,2018-12-31
4,2019,2019-01-01,2019-12-31
5,2020,2020-01-01,2020-08-31


https://www.practiceportuguese.com/learning-notes/seasons-of-the-year/
- Spring: 20 Mar < 21 June
- Summer: 21 June < 23 Sept
- Autumn: 23 Sept < 21 Dec
- Winter: 21 Dec < 20 Mar

In [36]:
# Though we can extract mm-dd from yyyy-mm-dd as below,
# this outputs as object dtype, which can't easily be compared to date parameters.
# df_main["arrival_date"].dt.strftime('%m-%d')
# So instead, use day of year:

df_main['day_of_year'] = df_main["arrival_date"].dt.dayofyear

def get_season(day_of_year):
    if 80 <= day_of_year <= 171:  # Mar 20 - Jun 20
        return 'spring'
    elif 172 <= day_of_year <= 266:  # Jun 21 - Sep 22
        return 'summer'
    elif 267 <= day_of_year <= 355:  # Sep 23 - Dec 20
        return 'autumn'
    else:
        return 'winter'

df_main['season'] = df_main['day_of_year'].apply(get_season)
# dataframe.apply(function) := apply a function along an axis of a dataframe

2015, 2018: partial summer, full autumn, partial winter <br>
2016, 2019: all seasons <br>
2017, 2020: partial winter, full spring, partial summer <br>

If we are to investigate cancellation *rate* and adr (*"rate"*), it is fine to include the partial seasons. <br>
I.e. we are not aggregating a sum, which would in that case be distorted by different coverage

In [37]:
# Cutom ordering for seasons
season_order = ["spring", "summer", "autumn", "winter"]
df_main['season'] = pd.Categorical(df_main['season'], categories=season_order, ordered=True)

# EDA - Univariate Analysis

In [38]:
df_main.groupby(df_main["arrival_date_year"]).size()

arrival_date_year
2015    12863
2016    41599
2017    31293
2018    12851
2019    54169
2020    30629
dtype: int64

In [39]:
# Sizes conditional on not cancelled
df_main[df_main["is_canceled"] == 0].groupby(["arrival_date_year"]).size()

arrival_date_year
2015    10160
2016    30400
2017    21187
2018    10160
2019    40279
2020    21187
dtype: int64

In [40]:
#From documentation, it is known that within data 2015 - 2017, only 2016 is in full, as the data runs from 01 July 2015 to 31 Aug 2017.
# Appears same for 2018 - 2020
df_main.groupby("arrival_date_year")["arrival_date_month"].nunique()

arrival_date_year
2015     6
2016    12
2017     8
2018     6
2019    12
2020     8
Name: arrival_date_month, dtype: int64

In [41]:
# To be sure, see min and max dates per year
df_main.groupby("arrival_date_year")["arrival_date"].agg(['min', 'max']).reset_index()

,arrival_date_year,min,max
0,2015,2015-07-01,2015-12-31
1,2016,2016-01-01,2016-12-31
2,2017,2017-01-01,2017-08-31
3,2018,2018-07-01,2018-12-31
4,2019,2019-01-01,2019-12-31
5,2020,2020-01-01,2020-08-31


In [42]:
# Would make sense to normalise to a per-month basis if comparing YoY (and in fact, YTD i.e. equivalent months)
df_months_per_year = df_main.groupby("arrival_date_year", as_index = False)["arrival_date_month"].nunique().rename(columns = {"arrival_date_month" : "count_months_covered"})
df_months_per_year

,arrival_date_year,count_months_covered
0,2015,6
1,2016,12
2,2017,8
3,2018,6
4,2019,12
5,2020,8


In [43]:
# Bookings per month by year:
df_size_by_year = df_main.groupby(df_main["arrival_date_year"], as_index = False).size()
df_size_by_year = df_size_by_year.merge(df_months_per_year, on = "arrival_date_year")
df_size_by_year["size_norm"] = df_size_by_year["size"] / df_size_by_year["count_months_covered"]
df_size_by_year

,arrival_date_year,size,count_months_covered,size_norm
0,2015,12863,6,2143.83
1,2016,41599,12,3466.58
2,2017,31293,8,3911.62
3,2018,12851,6,2141.83
4,2019,54169,12,4514.08
5,2020,30629,8,3828.62


In [44]:
# # Seems 2015 vs 2018; 2017 vs 2020 are just copies?
# df_main.groupby("arrival_date_year")["pax"].sum()
# df_main.groupby("arrival_date_year")["nights"].sum()

In [45]:
# # Below indicates the data across years are not copies
# cols_to_compare = [col for col in df_main.columns if col not in ["arrival_date_year", "arrival_date", "reservation_status", "arrival_date_month", "arrival_date_day_of_month", "arrival_date_week_number"]]

# df_2015 = df_main[df_main["arrival_date_year"] == 2015].sort_values(cols_to_compare).reset_index(drop=True)
# df_2018 = df_main[df_main["arrival_date_year"] == 2018].sort_values(cols_to_compare).reset_index(drop=True)

# print((df_2015[cols_to_compare]).equals(df_2018[cols_to_compare]))
# # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.equals.html

In [46]:
# Size split by hotel type
df_main.groupby(df_main["hotel"]).size()

hotel
City Hotel      110451
Resort Hotel     72953
dtype: int64

In [47]:
# Size split by cancellation
df_main.groupby(df_main["is_canceled"]).size()

is_canceled
0    133373
1     50031
dtype: int64

In [48]:
# Size split by deposit_type
df_main.groupby(df_main["deposit_type"]).size()

deposit_type
No Deposit    180948
Non Refund      2241
Refundable       215
dtype: int64

Early conceptual idea: investigate drivers of cancellations, especially if deposit type has impact

In [49]:
# Distribution by nights
df_main.aggregate({"nights" : ['min', 'max', 'mean', 'median', 'std']})

,nights
min,0.00
max,69.00
mean,3.68
median,3.00
std,2.74


In [50]:
Q1_nights = df_main['nights'].quantile(0.25)
Q3_nights = df_main['nights'].quantile(0.75)

IQR_nights = Q3_nights - Q1_nights

print(f"Q1: {Q1_nights}, Q3: {Q3_nights}, IQR: {IQR_nights}")

Q1: 2.0, Q3: 5.0, IQR: 3.0


In [51]:
# Whilst 69 nights is many std dev away from mean, it does seem plausible, just from an intuitive sense of hotel booking possibilities
# Other values at the tail support idea that this is not completely out of the blue:
df_main[df_main["nights"] > 21].groupby("nights").size()

nights
22    29
23    12
24    12
25    28
26    12
27     9
28    72
29    25
30    24
33     5
34     2
35    10
38     2
42     8
45     2
56     4
60     3
69     2
dtype: int64

Consider categorising nights stayed into custom short/medium/long stays

In [52]:
# Nights := number of nights guest did stay or booked to stay
# Note too that any instances of nights == 0 necessarily has is_canceled = 1
# Which may or may not be of business importance - best to check.
# Realistically, room probably can be booked for zero nights, so leave as is
# len(df_main.loc[(df_main["nights"] == 0) & (df_main["is_canceled"] == 0)])

len(df_main[((df_main["nights"] == 0) & (df_main["is_canceled"] == 0))])

0

In [53]:
# Distribution by pax
df_main.aggregate({"pax" : ['min', 'max', 'mean', 'median', 'std']})

,pax
min,1.00
max,55.00
mean,2.03
median,2.00
std,0.82


In [54]:
Q1_pax = df_main['pax'].quantile(0.25)
Q3_pax = df_main['pax'].quantile(0.75)

IQR_pax = Q3_pax - Q1_pax

print(f"Q1: {Q1_pax}, Q3: {Q3_pax}, IQR: {IQR_pax}")

Q1: 2.0, Q3: 2.0, IQR: 0.0


In [55]:
# 55 pax is mny std dev away from mean, but as with 'nights', is probably a situation where statistical norms take second place to business sense,
# and it does seem reasonable for a booking of 55 pax to emerge, especially from a 'Group' or 'Corporate' category.
# Would intend to check on this for a real stakeholder. For this project, leave as is
# Other values at the tail support idea that this is not completely out of the blue:
df_main.groupby("pax").size()

pax
1      33397
2     120813
3      20822
4       8040
5        282
6          3
10         6
12         5
20         6
26        15
27         6
40         3
50         3
55         3
dtype: int64

As with # nights, consider categorising pax in groups

In [56]:
# Other sizing and distributions
# df_main.groupby("distribution_channel")["pax"].agg(['mean', 'median', 'min', 'max'])
# df_main.groupby("customer_type")["pax"].agg(['mean', 'median', 'min', 'max'])
# df_main.groupby("market_segment")["pax"].agg(['mean', 'median', 'min', 'max'])
# df_main.groupby(["reserved_room_type"]).size()
# df_main.groupby(["assigned_room_type"]).size()

In [57]:
# Distribution by lead time
df_main.aggregate({"lead_time" : ['min', 'max', 'mean', 'median', 'std']})

,lead_time
min,0.00
max,709.00
mean,78.92
median,48.00
std,85.20


In [58]:
Q1_lead_time = df_main['lead_time'].quantile(0.25)
Q3_lead_time = df_main['lead_time'].quantile(0.75)

IQR_lead_time = Q3_lead_time - Q1_lead_time

print(f"Q1: {Q1_lead_time}, Q3: {Q3_lead_time}, IQR: {IQR_lead_time}")

Q1: 11.0, Q3: 123.0, IQR: 112.0


In [59]:
# Booking 709d in advance is quite far out, but still believable, and below suggests it is part of a legitimate long tail
df_main[df_main["lead_time"] >= 123].groupby(["lead_time"]).size()

lead_time
123    440
124    455
125    422
126    395
127    402
      ... 
615      4
622      4
626      2
629      4
709      2
Length: 355, dtype: int64

Likewise consider making categories to batch lead_times into manageable groups

TA = Travel Agents
TO = Tour Operators

In [60]:
df_main.groupby("market_segment").size()

market_segment
Aviation            446
Complementary       306
Corporate          9104
Direct            25207
Groups            10625
Offline TA/TO     30159
Online TA        107551
Undefined             6
dtype: int64

In [61]:
df_main.groupby(["distribution_channel"]).size()

distribution_channel
Corporate     10749
Direct        26766
GDS             355
TA/TO        145519
Undefined        15
dtype: int64

Agent := ID of travel angency making booking

In [62]:
# Note below examples of agent not null but other relevant columns not strictly being 'TA', which is what we would expect, given the terminology at hand
# ... not strictly intuitive what the business definitions here are - worth clarifying with SME. For now just leave as is (don't drop, etc.)
agent_not_null = df_main[df_main['agent'].notna()]
agent_not_null = agent_not_null[['distribution_channel', 'market_segment']].drop_duplicates().copy()
agent_not_null

,distribution_channel,market_segment
0,TA/TO,Online TA
1,TA/TO,Offline TA/TO
18,Direct,Direct
67,TA/TO,Direct
82,Direct,Groups
360,Corporate,Corporate
562,TA/TO,Groups
954,Corporate,Groups
2354,Undefined,Direct
2445,Undefined,Online TA


In [63]:
# There are 333 distinct agents
len(df_main["agent"].unique())

333

'Company' := ID of the company/entity that made the booking or responsible for paying the booking

In [64]:
# There are 346 distinct companies
len(df_main["company"].unique())

346

**Univariate Analysis Takeaways**

- There is expansive data / large ranges in interesting possible variables (probably independent) like pax, lead-time, market-segment, distribution channel, nights, cancellations
- Such that for numerical ones like nights and pax, may be adviseable to create categories for classifying batches
- As is probably expected, nights and pax have a tendency towards lower numbers (smaller bookings for fewer nights), but this does not mean an absence of larger, longer bookings

# Multiple Variate Analysis Ideas

1. **Cancellations** are a key metric to track. See across:
- Chronological time - note issue that there is a gap due to data collected having missing months
- Seasons (i.e. time of year, regardless of actual year)
- lead time - note this helps especially with the gap in the data
- market_segment, distribution_channel, customer_type, etc.

2. **adr** is another important outcome. May wish to investigate hypothesis of inverse correlation between adr and nights (longer stay = cheaper per night)
- Plus calculate actual revenue from booking, and investigate e.g. is per-period revenue up or down, and is this due to fewer bookings or less valuable ones?
- Hence **num bookings** becomes another metric

# Cancellations Analysis

- 'is_canceled' refers to hotel booking system
- 'reservation_status' refers to customer decision (canceled or no-show or checked-out already)
- Here, we will look into reservation_status == 'Canceled' --> has the customer announced a conscious decision to cancel?


https://experience-crm.fr/en/where-do-cancellations-come-from/#:~:text=Average%20cancellation%20rates&text=The%20average%20percentage%20of%20canceled,no%20room%20for%20nasty%20surprises.
- Published 2016-11-14
- Modified 2023-08-02
- The average percentage of canceled reservations, across **all sources**, is currently **24%**.
-  If you look only at **online** bookings, this rate increases to **38%**,
- and therefore all **“offline”** reservations [...] 10%.

https://hospitalitytech.com/global-cancellation-rate-hotel-reservations-reaches-40-average
- Published 2019-04-24; Modified 01/14/2025
- **online** distribution performance of more than 200 different channels for 680 properties in Europe between 2014 and 2018
- With the exception of 2018, every single channel has observed a marked increase in cancellation rate YoY.
- And, even in 2018, the number was 7.1 points above 2014.
- With a global average of almost **40%** cancellation rate [...].
- 2014 to 2018: 32.5%, 34.8%, 39.6%, 41.3%, 39.6%
- Across **online** sources: Booking Group, Expedia Group, Hotelbeds Group, HRS Group, Other OTAs, Other Wholesalers, Website Direct

https://www.phocuswire.com/One-in-five-hotel-bookings-on-the-web-are-cancelled
- Published 31 Dec, 2015
- Some **19%** of hotels that are booked **online** are cancelled before the guest arrives at the hotel, according to data provided by 

https://www.d-edge.com/avoid-guests-ghosting/
- According to our 2024 Hotel Distribution Report, hotel booking cancellation rates range from 18% to 42%.
- In Europe, the cancellation rate for direct bookings was 18% in 2023 [...] these rates increased for OTA bookings: in Europe, Expedia had a cancellation rate of 31%, while Booking Holdings reached 42%. 


In [ ]:
# 26.1% of bookings end up being canceled by the customer
# As illustrated above, channel is important to distinguishing / benchmarking performance
# With this limited perspective (considering all channels indiscriminately), we might say 26.1% is acceptable / comaprable to market standards, but not outstanding
# Although we might better treat the '40%' value as an outlier, and thus deem our cancellation rate over the benchmark, and in need of special attention
# (18% < 24% < 26.1% < ~40%)
len_canc = len(df_main[df_main["reservation_status"] == 'Canceled'])
len_total = len(df_main)

print(len_canc, len_total, round((len_canc / len_total), 3) * 100)

47852 183404 26.1


## Cancellation Rate over Time

In [ ]:
# Relatively lower cancellation rate in 2015, 2018 hints at systematically lower cancellation rates in Autumn and Winter.
# Good to get to the bottom of that, as a 11 p.p. swing from lowest to highest rate is fairly large and probably has revenue management implications
# The above is approx 50% relative increase, so fairly substantial
df_yyyy_cancellations = df_main.groupby("arrival_date_year").size().reset_index().rename(columns = {0 : "bookings"})

df_yyyy_cancellations2 = df_main[["arrival_date_year", "is_canceled"]].groupby("arrival_date_year")["is_canceled"].sum().reset_index().rename(columns = {"is_canceled" : "cancellations"})

df_yyyy_cancellations = df_yyyy_cancellations.merge(df_yyyy_cancellations2, on = "arrival_date_year")

df_yyyy_cancellations["cancellation_rate"] = df_yyyy_cancellations["cancellations"] / df_yyyy_cancellations["bookings"]
df_yyyy_cancellations = df_yyyy_cancellations[["arrival_date_year", "cancellation_rate", "bookings", "cancellations"]]
df_yyyy_cancellations

,arrival_date_year,cancellation_rate,bookings,cancellations
0,2015,0.21,12863,2703
1,2016,0.27,41599,11199
2,2017,0.32,31293,10106
3,2018,0.21,12851,2691
4,2019,0.26,54169,13890
5,2020,0.31,30629,9442


In [ ]:
# The above implies a year-effect on cancellation rate, however we must remember that certain years lack data from certain seasons.
# Notably, 2015 and 2018, 16/19, and 17/20 are paired in terms of seasonal coverage - their different cancellation rates might well be a seasonal effect
# Reminder:
    # 2015, 2018: partial summer, full autumn, partial winter; 2016, 2019: all seasons; 2017, 2020: partial winter, full spring, partial summer.
# As below, Spring higher and Autumn lower: this suggests a seasonal explanation to the relatively lower/higher cancellation rate in 2015 and 2018 / 2017 and 2020

df_seasonal_cancellations = df_main.groupby("season").size().reset_index().rename(columns = {0 : "bookings"})

df_seasonal_cancellations2 = df_main[["season", "is_canceled"]].groupby("season")["is_canceled"].sum().reset_index().rename(columns = {"is_canceled" : "cancellations"})

df_seasonal_cancellations = df_seasonal_cancellations.merge(df_seasonal_cancellations2, on = "season")

df_seasonal_cancellations["cancellation_rate"] = df_seasonal_cancellations["cancellations"] / df_seasonal_cancellations["bookings"]
df_seasonal_cancellations = df_seasonal_cancellations[["season", "cancellation_rate", "bookings", "cancellations"]]
df_seasonal_cancellations

C:\Users\danie\AppData\Local\Temp\ipykernel_22452\1460526646.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_seasonal_cancellations = df_main.groupby("season").size().reset_index().rename(columns = {0 : "bookings"})
C:\Users\danie\AppData\Local\Temp\ipykernel_22452\1460526646.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_seasonal_cancellations2 = df_main[["season", "is_canceled"]].groupby("season")["is_canceled"].sum().reset_index().rename(columns = {"is_canceled" : "cancellations"})


,season,cancellation_rate,bookings,cancellations
0,spring,0.29,47128,13740
1,summer,0.30,63464,19168
2,autumn,0.23,37773,8589
3,winter,0.24,35039,8534


In [ ]:
# Yet, it remains possible that this is in fact a year-effect,
# i.e. seasons have no inherent bias towards different cancellation rates, and it is merely the performances of particular year-seasons driving these results
# Or, a joint effect
# Either way, there is potential confounding

# There is not much evidence of a seasonal difference Spring vs Autumn when considering 2016 and 2019 data (the two 'complete' years):
    # 26% ( Spring) vs 24% and 28%
# Looking more broadly at the seasons though, it is generally true, per year, that Spring and Summer rates > Autumn and Winter.
# So, there might indeed be a seasonal impact to some extent, even if not super clear and definite

# We may have confidence in a year-effect:
# 2015 and 2018 have lower cancellation rates in general than 2016/2019, in turn lower than 2017 and 2020: generally tracks season-by-season

df_seasonal_cancellations = df_main.groupby(["season", "arrival_date_year"]).size().reset_index().rename(columns = {0 : "bookings"})

df_seasonal_cancellations2 = df_main[["season", "arrival_date_year", "is_canceled"]].groupby(["season", "arrival_date_year"])["is_canceled"].sum().reset_index().rename(columns = {"is_canceled" : "cancellations"})

df_seasonal_cancellations = df_seasonal_cancellations.merge(df_seasonal_cancellations2, on = ["arrival_date_year", "season"])

df_seasonal_cancellations["cancellation_rate"] = df_seasonal_cancellations["cancellations"] / df_seasonal_cancellations["bookings"]
df_seasonal_cancellations = df_seasonal_cancellations[["season", "arrival_date_year", "cancellation_rate", "bookings", "cancellations"]]
df_seasonal_cancellations.sort_values(by = ["season", "arrival_date_year"])

C:\Users\danie\AppData\Local\Temp\ipykernel_13020\1404188007.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_seasonal_cancellations = df_main.groupby(["season", "arrival_date_year"]).size().reset_index().rename(columns = {0 : "bookings"})
C:\Users\danie\AppData\Local\Temp\ipykernel_13020\1404188007.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_seasonal_cancellations2 = df_main[["season", "arrival_date_year", "is_canceled"]].groupby(["season", "arrival_date_year"])["is_canceled"].sum().reset_index().rename(columns = {"is_canceled" : "cancellations"})


,season,arrival_date_year,cancellation_rate,bookings,cancellations
0,spring,2015,NaN,0,0
1,spring,2016,0.26,11116,2922
2,spring,2017,0.33,12645,4140
3,spring,2018,NaN,0,0
4,spring,2019,0.26,11117,2927
5,spring,2020,0.31,12250,3751
6,summer,2015,0.25,6165,1526
7,summer,2016,0.30,12326,3679
8,summer,2017,0.36,10220,3710
9,summer,2018,0.25,6153,1514


In [81]:
# With decent confidence in year-effects (2015/18 < 2016/19 < 2017/20),
# and some suspicion of seasonal effects too,
# would probably be best to build a classification model e.g. logistic that breaks down the relative contribution of both effects.

# That said, time-based analysis is not really the end-goal.
# Instead, actually monitarable hence actionable independent variables (market segment, distribution channel, deposit type, lead time) and how they impact cancellation rates.
# A simple approach (don't build model) to assess the impact of categories/values within these more interesting independent variables is to just do this per period.
    # E.g. in 2015-Winter, how did the market segments compare? What about Spring 2016? Etc repeat as needed.
    # In an ideal case, in each and every period, market segment X would have higher rate --> can conclude something definit about this segment
    # Also, might be able to deduce that the period-effect is in fact a segment-effect, e.g. in Autumn 2017, the cancellation rate of segment F shot up (and was of significant magnitude)
    # If all segments rise and fall in accordance with observed period-effects, can even get away with suggesting this effect is shared by segments, therefore not a confounding factor (with caution...)

# Still, scientifically finding coefficients for year and season effects would help especially if doing an across-time analysis (not constrained to period-by-period views),
# as it would enable appropriate weighting of segments based on relative seasonal/yyyy-seasonal representation

In [79]:
# # Consistently, the City Hotel has a higher cancellation rate than the Resort Hotel,
# # and the seasonal effect is also observed again
# df_seasonal_cancellations_by_hotel = df_main.groupby(["season", "hotel"]).size().reset_index().rename(columns = {0 : "bookings"})

# df_seasonal_cancellations_by_hotel2 = df_main[["season", "hotel", "is_canceled"]].groupby(["season", "hotel"])["is_canceled"].sum().reset_index().rename(columns = {"is_canceled" : "cancellations"})

# df_seasonal_cancellations_by_hotel = df_seasonal_cancellations_by_hotel.merge(df_seasonal_cancellations_by_hotel2, on = ["season", "hotel"])

# df_seasonal_cancellations_by_hotel["cancellation_rate"] = df_seasonal_cancellations_by_hotel["cancellations"] / df_seasonal_cancellations_by_hotel["bookings"]

# df_seasonal_cancellations_by_hotel = df_seasonal_cancellations_by_hotel[["season", "hotel", "cancellation_rate", "bookings", "cancellations"]].sort_values(by = "season")
# df_seasonal_cancellations_by_hotel

In [82]:
# ignore below need to do by year-season

df_seasonal_cancellations_by_distrib = df_main.groupby(["season", "distribution_channel"]).size().reset_index().rename(columns = {0 : "bookings"})

df_seasonal_cancellations_by_distrib2 = df_main[["season", "distribution_channel", "is_canceled"]].groupby(["season", "distribution_channel"])["is_canceled"].sum().reset_index().rename(columns = {"is_canceled" : "cancellations"})

df_seasonal_cancellations_by_distrib = df_seasonal_cancellations_by_distrib.merge(df_seasonal_cancellations_by_distrib2, on = ["season", "distribution_channel"])

df_seasonal_cancellations_by_distrib["cancellation_rate"] = df_seasonal_cancellations_by_distrib["cancellations"] / df_seasonal_cancellations_by_distrib["bookings"]

df_seasonal_cancellations_by_distrib = df_seasonal_cancellations_by_distrib[["season", "distribution_channel", "cancellation_rate", "bookings", "cancellations"]].sort_values(by = ["season", "distribution_channel"])
df_seasonal_cancellations_by_distrib

df_seasonal_cancellations_by_mrkt = df_main.groupby(["season", "market_segment"]).size().reset_index().rename(columns = {0 : "bookings"})

df_seasonal_cancellations_by_mrkt2 = df_main[["season", "market_segment", "is_canceled"]].groupby(["season", "market_segment"])["is_canceled"].sum().reset_index().rename(columns = {"is_canceled" : "cancellations"})

df_seasonal_cancellations_by_mrkt = df_seasonal_cancellations_by_mrkt.merge(df_seasonal_cancellations_by_mrkt2, on = ["season", "market_segment"])

df_seasonal_cancellations_by_mrkt["cancellation_rate"] = df_seasonal_cancellations_by_mrkt["cancellations"] / df_seasonal_cancellations_by_mrkt["bookings"]

df_seasonal_cancellations_by_mrkt = df_seasonal_cancellations_by_mrkt[["season", "market_segment", "cancellation_rate", "bookings", "cancellations"]].sort_values(by = ["season", "market_segment"])
df_seasonal_cancellations_by_mrkt

# Consistently, the City Hotel has a higher cancellation rate than the Resort Hotel,
# and the arrival_date_yearal effect is also observed again
df_arrival_date_yearal_cancellations_by_hotel = df_main.groupby(["arrival_date_year", "hotel"]).size().reset_index().rename(columns = {0 : "bookings"})

df_arrival_date_yearal_cancellations_by_hotel2 = df_main[["arrival_date_year", "hotel", "is_canceled"]].groupby(["arrival_date_year", "hotel"])["is_canceled"].sum().reset_index().rename(columns = {"is_canceled" : "cancellations"})

df_arrival_date_yearal_cancellations_by_hotel = df_arrival_date_yearal_cancellations_by_hotel.merge(df_arrival_date_yearal_cancellations_by_hotel2, on = ["arrival_date_year", "hotel"])

df_arrival_date_yearal_cancellations_by_hotel["cancellation_rate"] = df_arrival_date_yearal_cancellations_by_hotel["cancellations"] / df_arrival_date_yearal_cancellations_by_hotel["bookings"]

df_arrival_date_yearal_cancellations_by_hotel = df_arrival_date_yearal_cancellations_by_hotel[["arrival_date_year", "hotel", "cancellation_rate", "bookings", "cancellations"]].sort_values(by = "arrival_date_year")
df_arrival_date_yearal_cancellations_by_hotel

df_arrival_date_yearal_cancellations_by_distrib = df_main.groupby(["arrival_date_year", "distribution_channel"]).size().reset_index().rename(columns = {0 : "bookings"})

df_arrival_date_yearal_cancellations_by_distrib2 = df_main[["arrival_date_year", "distribution_channel", "is_canceled"]].groupby(["arrival_date_year", "distribution_channel"])["is_canceled"].sum().reset_index().rename(columns = {"is_canceled" : "cancellations"})

df_arrival_date_yearal_cancellations_by_distrib = df_arrival_date_yearal_cancellations_by_distrib.merge(df_arrival_date_yearal_cancellations_by_distrib2, on = ["arrival_date_year", "distribution_channel"])

df_arrival_date_yearal_cancellations_by_distrib["cancellation_rate"] = df_arrival_date_yearal_cancellations_by_distrib["cancellations"] / df_arrival_date_yearal_cancellations_by_distrib["bookings"]

df_arrival_date_yearal_cancellations_by_distrib = df_arrival_date_yearal_cancellations_by_distrib[["arrival_date_year", "distribution_channel", "cancellation_rate", "bookings", "cancellations"]].sort_values(by = ["arrival_date_year", "distribution_channel"])
df_arrival_date_yearal_cancellations_by_distrib

## Cancellations by segment/channel

- https://www.siteminder.com/r/hotel-distribution-concepts-best-practice/
- A hotel distribution channel can be any method or platform by which your hotel sells its rooms.

- vs market_segment = customer profiling

- E.g. 'direct' market_segment refers to customer booking directly for themselves, as opposed to for a group tour or corporate,<br>whilst 'direct' distribution_channel refers to the rooms having been booked via the hotel website / walk-in / similar direct action

- For whatever reason, distribution_channel data here doesn't distinguish offline/online, but can still try seeing any patterns across the existing channels given

df_main[['distribution_channel', 'market_segment']].drop_duplicates().sort_values(by = ["distribution_channel", "market_segment"]).reset_index(drop = True)


In [83]:
# Ignore below needs revision

# Revenue Analysis

## Revenue Analysis Prelim

# Revenue associated with booking of rooms
df_main["room_revenue"] = df_main["nights"] * df_main["adr"]
df_main[["room_revenue", "nights", "adr"]].head(10)

# Revenue associated with meal booked
# Assume that 'SC' meals (self-catered) are not priced in error, despite somehow being more expensive that full-board FB meals
# Assume meals apply per day per non-baby pax - unlikely, but there is a lack of information on exact approach used
    #  Whereby days = nights + 1
df_meal_cost

df_main = pd.merge(df_main, df_meal_cost, on = "meal", how = "left").merge(df_market_segment, on = "market_segment").rename(columns = {"Cost" : "meal_cost", "Discount" : "discount"})
df_main.head(10)


df_main["meal_revenue"] = (df_main.nights + 1) * df_main.meal_cost * (df_main.pax - df_main.babies)
df_main[["meal_revenue", "nights", "pax", "babies", "meal_cost"]].head(10)


df_main.groupby("deposit_type").size()

# Assume discount applied to overall price of booking
# Assume no revenue at all collected for cancellation if deposit_type in ("No Deposit", "Refundable"), but 100% kept if it was "Non Refund"

# Although booking revenue assumed zero for cancelled, still calculate the would-be revenue, to enable a view on unmaterialised / lost revenue
# Just filter out later in actual analysis

df_main["booking_revenue"] = (df_main.meal_revenue + df_main.room_revenue) * (1 - df_main.discount)

df_main[["is_canceled", "booking_revenue", "meal_revenue", "room_revenue", "discount"]].head(10)

# if df_main["is_canceled"] == 1:
#     df_main["booking_revenue"] = 0
# elif df_main["is_canceled"] == 0:
#     df_main["booking_revenue"] = round(((df_main.meal_revenue + df_main.room_revenue) * (1 - df_main.discount)), 2)
# Above doesn't work as the if conditions produce a series (each row - is the condition True or False?),
# but we instead want to be processing the logic row by row

# df_main["booking_revenue"] = np.where(df_main["is_canceled"] == 1, 0, (df_main.meal_revenue + df_main.room_revenue) * (1 - df_main.discount))
# df_main["booking_revenue"] = round(df_main["booking_revenue"], 2)


df_main_complete = df_main[(df_main.is_canceled == 0) | ((df_main.is_canceled == 1) & (df_main.deposit_type == "Non Refund"))]
df_main_cancelled = df_main[((df_main.is_canceled == 1) & (df_main.deposit_type != "Non Refund"))]
print(len(df_main_complete))
print(len(df_main_cancelled))
print(len(df_main))


## Revenue Analysis 1 - Overview

### Direct comparison

df_main_complete.groupby("arrival_date_year", as_index = False)["booking_revenue"].sum()

Naturally, 2020 would appear to be lagging behind due to incomplete year. Best to compare YTD and revenue per month

### YTD 2019 vs 2020

# Check to be sure we end on August 31, and don't have to instead select a particular day
df_main[(df_main.arrival_date_month == 8) &(df_main.arrival_date_year == 2020)].groupby("arrival_date_month")["arrival_date_day_of_month"].max()

df_2019_vs_2020 = df_main_complete[(df_main_complete.arrival_date_month < 9) & df_main_complete.arrival_date_year.between(2019, 2020)].reset_index(drop = True)
df_YTD_2020_revenue = df_2019_vs_2020.groupby(["arrival_date_year"])["booking_revenue"].sum().reset_index().rename(columns = {"arrival_date_year" : "YTD"})
df_YTD_2020_revenue

print(df_YTD_2020_revenue["booking_revenue"].loc[1] - df_YTD_2020_revenue["booking_revenue"].loc[0])
print((df_YTD_2020_revenue["booking_revenue"].loc[1] - df_YTD_2020_revenue["booking_revenue"].loc[0]) / df_YTD_2020_revenue["booking_revenue"].loc[0])

# The % change in equivalent time period 2020 vs 2019 is -1.2%
# Arguably this is not a huge issue, but depends on context etc. Investigate more later

### YTD 2018 vs 2019

# Check to be sure we start on July 01, and don't have to instead select a particular day

print(df_main["arrival_date_month"][(df_main.arrival_date_year == 2018)].min())

df_main[(df_main.arrival_date_month == 7) &(df_main.arrival_date_year == 2018)].groupby("arrival_date_month")["arrival_date_day_of_month"].min()

df_2018_vs_2019 = df_main_complete[(df_main.arrival_date_month > 7) & df_main.arrival_date_year.between(2018, 2019)].reset_index(drop = True)
df_YTD_2018_revenue = df_2018_vs_2019.groupby(["arrival_date_year"])["booking_revenue"].sum().reset_index().rename(columns = {"arrival_date_year" : "YTD"})
df_YTD_2018_revenue

print(df_YTD_2018_revenue["booking_revenue"].loc[1] - df_YTD_2018_revenue["booking_revenue"].loc[0])
print((df_YTD_2018_revenue["booking_revenue"].loc[1] - df_YTD_2018_revenue["booking_revenue"].loc[0]) / df_YTD_2018_revenue["booking_revenue"].loc[0])

# The % change in equivalent time period 2019 vs 2018 is +154.1%

### Revenue per Month from completed bookings, 2018 - 2020

df_revenue = df_main_complete.groupby(["arrival_date_year", "arrival_date_month", "hotel"], as_index = False)["booking_revenue"].sum().rename(columns = {"booking_revenue" : "revenue_per_ym_by_type"})
df_revenue = df_revenue.merge(df_months_per_year, on = "arrival_date_year")

# Split by hotel type:
df_revenue["revenue_per_y_by_type"] = df_revenue.groupby(["arrival_date_year", "hotel"])["revenue_per_ym_by_type"].transform("sum")
df_revenue["avg_revenue_per_m_by_y_by_type"] = round(((df_revenue["revenue_per_y_by_type"] / df_revenue["count_months_covered"])), 2)

# Irrespective of hotel type:
df_revenue["revenue_per_ym"] = df_revenue.groupby(["arrival_date_year", "arrival_date_month"])["revenue_per_ym_by_type"].transform("sum")
df_revenue["revenue_per_y"] = df_revenue.groupby("arrival_date_year")["revenue_per_ym_by_type"].transform("sum")
df_revenue["avg_revenue_per_m_by_y"] = round(((df_revenue["revenue_per_y"] / df_revenue["count_months_covered"])), 2)

# (Check sums before col drop)
df_revenue.drop(["count_months_covered", "revenue_per_y", "revenue_per_y_by_type"], axis = 1, inplace=True)
df_revenue

df_revenue_agg = df_revenue[["arrival_date_year", "avg_revenue_per_m_by_y"]].drop_duplicates().reset_index(drop = True)
df_revenue_agg

# Pandas chart
# pd_bar_1 = df_revenue_agg.plot(kind = "bar", x = "arrival_date_year", y = "avg_revenue_per_m_by_y", figsize = (4, 6), title = "Average Revenue per Month by Year", legend = None, rot = 0, xlabel = "Year", ylabel = "USD (M)")

# pd_bar_1.get_yaxis().get_offset_text().set_visible(False)
# # Suppress the '1e6' label top left

ax = plt.subplots(figsize=(4, 6))[1]
# plt.subplots() creates tuple with (figure object, axes object), so we assign 'ax' to be the axes object
# In another instance, we may instead have wanted to done assigment: fig, ax = plt.subplots(), therefore assigning fig = figure object and ax = axes object
ax.get_yaxis().get_offset_text().set_visible(False)
# suppress the '1e6' top left label
plt.bar(df_revenue_agg['arrival_date_year'], df_revenue_agg['avg_revenue_per_m_by_y'])
plt.xticks(df_revenue_agg['arrival_date_year'])
plt.xlabel("Year")
plt.title("Average Revenue per Month by Year")
plt.ylabel("USD (M)")
# plt.grid(visible = True, axis = "y")
plt.show()


We now see graphically that average revenue from completed bookings per month was comparable in 2019 and 2020, but dramatically less in 2018

Revenue Analysis Section 1 wrap-up:<br>
1) Avg revenue per month grew drastically 2018 - 19.<br>
2) Avg revenue per month then declined slightly YoY 2020

To investigate:
- Drivers of growth 2019 vs 2018.
- Did these same drivers maintain in 2020, or did different combinations come into play?
- Attempt to drive insights into whether 2019 and 2020 high relative performance are sustainable or volatile

## Revenue Analysis 2 - Drivers 2019 vs 2018

### Diagnose number of (completed) bookings vs value of (completed) bookings

df_2018_bookings = df_2018_vs_2019.groupby(["arrival_date_year", "arrival_date_month"]).size().reset_index(name="bookings")

# df_2018_bookings = df_2018_bookings.merge(df_revenue, on = ["arrival_date_year", "arrival_date_month"])[["arrival_date_year", "arrival_date_month", "revenue_per_ym", "bookings"]].drop_duplicates()
# df_2018_bookings["revenue_per_booking"] = df_2018_bookings.revenue_per_ym / df_2018_bookings.bookings
# df_2018_bookings = df_2018_bookings.pivot(index = "arrival_date_month", columns = "arrival_date_year", values = ["bookings", "revenue_per_booking"])
# df_2018_bookings[("bookings", "num_bookings_delta")] = df_2018_bookings[("bookings", 2019)] - df_2018_bookings[("bookings", 2018)]
# df_2018_bookings[("revenue_per_booking", "booking_rev_delta")] = df_2018_bookings[("revenue_per_booking", 2019)] - df_2018_bookings[("revenue_per_booking", 2018)]
# df_2018_bookings = df_2018_bookings.sort_index(axis=1, level=0)

df_2018_bookings_pivot = df_2018_bookings.pivot(index = "arrival_date_month", columns = "arrival_date_year", values = "bookings")
df_2018_bookings_pivot.columns.name = "bookings"
df_2018_bookings_pivot["abs_delta"] = df_2018_bookings_pivot[2019] - df_2018_bookings_pivot[2018]
df_2018_bookings_pivot["pc_delta"] = (df_2018_bookings_pivot[2019] - df_2018_bookings_pivot[2018]) / df_2018_bookings_pivot[2018]
df_2018_bookings_pivot.loc["YTD", [2018, 2019, "abs_delta"]] = df_2018_bookings_pivot.sum()
df_2018_bookings_pivot.loc["YTD", ["pc_delta"]] = df_2018_bookings_pivot.mean()
df_2018_bookings_pivot

The revenue gain 2019 vs 2018 is driven strongly by more bookings

df_2018_bookings_rev = df_2018_vs_2019.groupby(["arrival_date_year", "arrival_date_month"])["booking_revenue"].sum().reset_index(name = "booking_revenue")

df_2018_bookings_rev = df_2018_bookings_rev.merge(df_2018_bookings, on = ["arrival_date_year", "arrival_date_month"])
# df_2018_bookings_rev # check columns ok

df_2018_bookings_rev["revenue_per_booking"] = df_2018_bookings_rev.booking_revenue / df_2018_bookings.bookings
df_2018_bookings_rev = df_2018_bookings_rev[["arrival_date_year", "arrival_date_month", "revenue_per_booking"]]

df_2018_bookings_rev_pivot = df_2018_bookings_rev.pivot(index = "arrival_date_month", columns = "arrival_date_year", values = "revenue_per_booking")
df_2018_bookings_rev_pivot.columns.name = "revenue_per_booking"

df_2018_bookings_rev_pivot["abs_delta"] = df_2018_bookings_rev_pivot[2019] - df_2018_bookings_rev_pivot[2018]
df_2018_bookings_rev_pivot["pc_delta"] = (df_2018_bookings_rev_pivot[2019] - df_2018_bookings_rev_pivot[2018]) / df_2018_bookings_rev_pivot[2018]
df_2018_bookings_rev_pivot.loc["YTD", [2018, 2019]] = df_2018_bookings_rev_pivot.sum()
df_2018_bookings_rev_pivot.loc["YTD", ["pc_delta", "abs_delta"]] = df_2018_bookings_rev_pivot.mean()

df_2018_bookings_rev_pivot

The revenue gain 2019 vs 2018 is driven weakly by more valuable bookings

As the 2019 vs 2018 difference is more obviously driven by number of bookings, rather than value of bookings, prioritise looking into this aspect

### Investigate drivers of number of (completed / non-refunded) bookings

#### By Hotel Type

df_2018_bookings_hotel = df_2018_vs_2019.groupby(["hotel", "arrival_date_year", "arrival_date_month"]).size().reset_index(name = "bookings")
# df_bookings_hotel
df_2018_bookings_hotel_pivot = df_2018_bookings_hotel.pivot(index = "arrival_date_month", columns = ["hotel", "arrival_date_year"], values = "bookings")

df_2018_bookings_hotel_pivot[("City Hotel", "abs_delta")] = df_2018_bookings_hotel_pivot[("City Hotel", 2019)] - df_2018_bookings_hotel_pivot[("City Hotel", 2018)]
df_2018_bookings_hotel_pivot[("Resort Hotel", "abs_delta")] = df_2018_bookings_hotel_pivot[("Resort Hotel", 2019)] - df_2018_bookings_hotel_pivot[("Resort Hotel", 2018)]
df_2018_bookings_hotel_pivot[("City Hotel", "pc_delta")] = (df_2018_bookings_hotel_pivot[("City Hotel", 2019)] - df_2018_bookings_hotel_pivot[("City Hotel", 2018)]) / df_2018_bookings_hotel_pivot[("City Hotel", 2018)]
df_2018_bookings_hotel_pivot[("Resort Hotel", "pc_delta")] = (df_2018_bookings_hotel_pivot[("Resort Hotel", 2019)] - df_2018_bookings_hotel_pivot[("Resort Hotel", 2018)]) / df_2018_bookings_hotel_pivot[("Resort Hotel", 2018)]
df_2018_bookings_hotel_pivot = df_2018_bookings_hotel_pivot.sort_index(axis=1, level=0)

df_2018_bookings_hotel_pivot.loc["YTD", [("City Hotel", 2018), ("City Hotel", 2019), ("City Hotel", "abs_delta"), ("Resort Hotel", 2018), ("Resort Hotel", 2019), ("Resort Hotel", "abs_delta") ]] = df_2018_bookings_hotel_pivot.sum()
df_2018_bookings_hotel_pivot.loc["YTD", [("City Hotel", "pc_delta"), ("Resort Hotel", "pc_delta")]] = df_2018_bookings_hotel_pivot.mean()

df_2018_bookings_hotel_pivot

Both hotel types contributed strongly to the increased number of bookings 2019 vs 2018.

#### By market segment

# df_main[["market_segment", "distribution_channel"]].
# unique can't apply to a dataframe (only a column (series) within)
# So:
df_main[["market_segment", "distribution_channel"]].drop_duplicates().sort_values(by = "market_segment").reset_index(drop = True)

# df_main_trunc = df_main[["arrival_date_year", "arrival_date_month", "market_segment", "distribution_channel", "booking_revenue", "meal_revenue", "room_revenue", "discount"]][(df_main.is_canceled == 0) & (df_main.arrival_date_month < 9) & df_main.arrival_date_year.between(2019, 2020)].reset_index(drop = True)
# df_main_trunc.head(10)

df_2018_segments = df_2018_vs_2019.groupby(["market_segment", "arrival_date_year"]).size().reset_index(name = "bookings").sort_values(by = ["arrival_date_year", "market_segment"])
df_2018_segments_pivot = df_2018_segments.pivot(index = "market_segment", columns = "arrival_date_year", values = "bookings")
df_2018_segments_pivot.fillna(0, inplace = True)
df_2018_segments_pivot.fillna(0, inplace = True)
df_2018_segments_pivot["abs_delta"] = df_2018_segments_pivot[2019] - df_2018_segments_pivot[2018]
df_2018_segments_pivot["pc_delta"] = (df_2018_segments_pivot[2019] - df_2018_segments_pivot[2018]) / df_2018_segments_pivot[2018]
df_2018_segments_pivot

Market segments across the board contributed to the increased number of bookings 2018 vs 2019.<br>
Considering scale and % delta, "**Online TA**" was the most critical driver, followed by "Offline TA/TO" and "Direct".<br>
That said, high % delta increase was realised by segments across the board

## Revenue Analysis 3 - Drivers 2020 vs (2019 vs 2018)

I.e. we know the drivers of stronger performance in 2019 over 2018 were:
1) More bookings through both hotel types, arguably more so with City Hotels
2) More bookings through all market segments, particular attention due for "**Online TA**", "Offline TA/TO" and "Direct"

Meanwhile, these were not particular strong drivers:
1) Value per bookings

So: given that 2020 YTD revenue performance has almost matched 2019,<br>
want to investigate behaviours going into 2020.<br>
E.g. market segments: did those key market segments continue to grow? Are there things to note about lead time, rate of cancellation, etc.?
E.g. hotels: has the same ratio of city:hotel maintained?

But first: see month by month shape of 2020 vs 2019 - look for flags e.g. was 'matching' YoY performance driven by extreme highs at first, then dampened, and therefore we should not expect EoY matching performance?

### Revenue Shape

df_2020_shape = df_2019_vs_2020.groupby(["arrival_date_year", "arrival_date_month"], as_index = False)["booking_revenue"].sum()
df_2020_shape_pivot = df_2020_shape.pivot(index = "arrival_date_month", columns = "arrival_date_year", values = "booking_revenue")
df_2020_shape_pivot.columns.name = "booking_revenue"
df_2020_shape_pivot

Revenue shows no signs of stopping 2020 YTD, in fact it is monotonically rising

### Market Segment Behaviour

#### Number of Bookings

df_2020_segments = df_2019_vs_2020.groupby(["market_segment", "arrival_date_year"]).size().reset_index(name = "bookings").sort_values(by = ["arrival_date_year", "market_segment"])
df_2020_segments_pivot = df_2020_segments.pivot(index = "market_segment", columns = "arrival_date_year", values = "bookings")
df_2020_segments_pivot.fillna(0, inplace = True)
df_2020_segments_pivot["abs_delta"] = df_2020_segments_pivot[2020] - df_2020_segments_pivot[2019]
df_2020_segments_pivot["pc_delta"] = (df_2020_segments_pivot[2020] - df_2020_segments_pivot[2019]) / df_2020_segments_pivot[2019]
df_2020_segments_pivot

df_2020_segments_prop = df_2019_vs_2020.groupby(["market_segment", "arrival_date_year"]).size().reset_index(name = "bookings").sort_values(by = ["arrival_date_year", "market_segment"])
df_2020_segments_prop_pivot = df_2020_segments_prop.pivot(index = "market_segment", columns = "arrival_date_year", values = "bookings")
df_2020_segments_prop_pivot["2019_total"] = df_2020_segments_prop_pivot[2019].sum()
df_2020_segments_prop_pivot["2019_pc"] = df_2020_segments_prop_pivot[2019] / df_2020_segments_prop_pivot["2019_total"]
df_2020_segments_prop_pivot["2020_total"] = df_2020_segments_prop_pivot[2020].sum()
df_2020_segments_prop_pivot["2020_pc"] = df_2020_segments_prop_pivot[2019] / df_2020_segments_prop_pivot["2020_total"]
df_2020_segments_prop_pivot = df_2020_segments_prop_pivot[[2019, "2019_total", "2019_pc", 2020, "2020_total", "2020_pc"]]
df_2020_segments_prop_pivot.columns = pd.MultiIndex.from_tuples([("2019", "Count"), ("2019", "Total"), ("2019", "pc"), ("2020", "Count"), ("2020", "Total"), ("2020", "pc")])
df_2020_segments_prop_pivot

- For 2020 YTD vs 2019, the proportion of total number of bookings accountable to each market segment was very comparable
- Especially encouraging to see key segment "Online TA" hold strong, as well as "Direct" (slight losses in 2020 YTD but close to matching 2019 YTD)
- However, in df_2020_segments_pivot we see that secondary key segment "Offline TA/TO" decreased 36% in 2020 YTD versus 2019 YTD
    - This worsened performance of this segment was a large contributor to the overall shortfall in bookings in 2020 YTD.<br>
    Calculated elsewhere: it is worth 92% of the overall decline in number of bookings
    - As Offline TA/TO commands approx 23% of leads, is worth bolstering support for this


#### Cancellations

First see overall proportion of bookings per year (to-date) cancelled, regardless of refundability

# df_main_complete := actually completed or cancelled + non-refundable
# df_main := agnostic to refundability

# Filter down df_main to appropriate dates and useful cols:

df_2019_vs_2020_canc = df_main[["arrival_date_year", "market_segment", "is_canceled", "deposit_type"]][(df_main.arrival_date_month < 9) & df_main.arrival_date_year.between(2019, 2020)].reset_index(drop = True)

# Calculate % of bookings per year (to-date) cancelled, regardless of refundability

df_2020_segments_canc = df_2019_vs_2020_canc.groupby(["arrival_date_year", "market_segment"])["is_canceled"].sum().reset_index().rename(columns = {"is_canceled" : "num_cancelled"})
df_2020_segments_canc

df_2020_segments_comp = df_2019_vs_2020_canc[df_2019_vs_2020_canc.is_canceled == 0].groupby(["arrival_date_year", "market_segment"]).size().reset_index().rename(columns = {0 : "num_complete"})
df_2020_segments_comp

df_2020_segments_comp = df_2020_segments_comp.merge(df_2020_segments_canc, on = ["arrival_date_year", "market_segment"])
df_2020_segments_comp["booked"] = df_2020_segments_comp["num_complete"] + df_2020_segments_comp["num_cancelled"]
df_2020_segments_comp["pc_cancelled"] =  df_2020_segments_comp["num_cancelled"] / df_2020_segments_comp["booked"]
df_2020_segments_comp = df_2020_segments_comp[["arrival_date_year", "market_segment", "num_cancelled", "pc_cancelled"]]
df_2020_segments_comp_pivot = df_2020_segments_comp.pivot(index = "market_segment", columns = "arrival_date_year", values = ["num_cancelled", "pc_cancelled"])
# df_2020_segments_comp_pivot.columns.name = "cancellation analysis"
df_2020_segments_comp_pivot[("pc_cancelled", "delta p.p.")] = df_2020_segments_comp_pivot[("pc_cancelled", 2020)] - df_2020_segments_comp_pivot[("pc_cancelled", 2019)]
df_2020_segments_comp_pivot[("num_cancelled", "delta")] = df_2020_segments_comp_pivot[("num_cancelled", 2020)] - df_2020_segments_comp_pivot[("num_cancelled", 2019)]
df_2020_segments_comp_pivot[("num_cancelled", "pc_delta")] = df_2020_segments_comp_pivot[("num_cancelled", "delta")] / df_2020_segments_comp_pivot[("num_cancelled", 2019)]

df_2020_segments_comp_pivot = df_2020_segments_comp_pivot.sort_index(axis=1, level=0)
df_2020_segments_comp_pivot

Overall there are more cancelled bookings in 2020 YTD, and this is especially driven by Online TA segment.<br>
Calculated elsewhere, 1984/1870 = 106% of the increase in cancellations in 2020 was due to this segment.<br>
The scale of numbers implies this might naturally be a segment with greater propensity to cancel (ease of online facilities?),<br>
but still, the proportion of cancellations went up 6 p.p. Even if this segment naturally cancels more, should still exercise controls over the proportion doing so.

Next, analyse with nuance: some cancellations still generate revenue.<br>
... Assess 'quality' of completed bookings, i.e. if generates revenue, it is a quality booking

# What % of bookings per year (to-date) were 'completed' i.e. generated revenue, i.e. actually completed, or non-refundable?
# Reminder: df_2019_vs_2020 is already filtered on date (YTD) and either actually completed, or cancelled but non-refundable

df_2020_segments_valuable = df_2019_vs_2020.groupby(["arrival_date_year", "market_segment"]).size().reset_index().rename(columns = {0 : "num_generating_revenue"})

df_2020_segments_non_valuable = df_main_cancelled.groupby(["arrival_date_year", "market_segment"]).size().reset_index().rename(columns = {0 : "num_not_generating_revenue"})

df_2020_segments_valuable = df_2020_segments_non_valuable.merge(df_2020_segments_valuable, on = ["arrival_date_year", "market_segment"])

df_2020_segments_valuable["booked"] = df_2020_segments_valuable["num_generating_revenue"] + df_2020_segments_valuable["num_not_generating_revenue"]
df_2020_segments_valuable["pc_generating_revenue"] = df_2020_segments_valuable["num_generating_revenue"] / df_2020_segments_valuable["booked"]

df_2020_segments_valuable = df_2020_segments_valuable[["arrival_date_year", "market_segment", "pc_generating_revenue"]]
df_2020_segments_valuable
df_2020_segments_valuable_pivot = df_2020_segments_valuable.pivot(index = "market_segment", columns = "arrival_date_year", values = "pc_generating_revenue")
df_2020_segments_valuable_pivot.columns.name = "% generating revenue"
df_2020_segments_valuable_pivot["delta p.p."] = df_2020_segments_valuable_pivot[2020] - df_2020_segments_valuable_pivot[2019]
df_2020_segments_valuable_pivot

The % of bookings which generated revenue increased across all segment types, which is encouraging.<br>
Though Online TA had many cancellations, at least those that went through were of higher quality, arguably(*).<br>
(*): Bookings that generated revenue through capture of non-refundable deposit are arguably non-sutainable / based on luck. But not necessarily - depends. Whatever the case, analyse next:

# What proportion of revenue-generating bookings per segment were categorised as such due to being non-refundable cancellations as opposed to actual completions?
# Reminder: df_2019_vs_2020 is already filtered on date (YTD) and either actually completed, or cancelled but non-refundable
# I.e. analysis is strictly for revenue-generating bookings, not bookings in YTD overall
df_2020_segments_cancel = df_2019_vs_2020.groupby(["arrival_date_year", "market_segment"])["is_canceled"].sum().reset_index().rename(columns = {"is_canceled" : "num_cancelled"})
# How many of the overall revenue-generating bookings per category (year, segment) were cancelled (generated revenue through capture of non-refundable deposit)?
df_2020_segments_size = df_2019_vs_2020.groupby(["arrival_date_year", "market_segment"]).size().reset_index().rename(columns = {0 : "size"})
# size := number of bookings per category (year, segment) that generated revenue
df_2020_segments_size = df_2020_segments_cancel.merge(df_2020_segments_size, on = ["arrival_date_year", "market_segment"])

df_2020_segments_size["pc_cancelled"] = df_2020_segments_size["num_cancelled"] / df_2020_segments_size["size"]

df_2020_segments_size = df_2020_segments_size[["arrival_date_year", "market_segment", "pc_cancelled"]]
df_2020_segments_size_pivot = df_2020_segments_size.pivot(index = "market_segment", columns = "arrival_date_year", values = "pc_cancelled")
df_2020_segments_size_pivot.columns.name = "pc cancelled"
df_2020_segments_size_pivot["delta p.p."] = df_2020_segments_size_pivot[2020] - df_2020_segments_size_pivot[2019]
df_2020_segments_size_pivot

The increase in proportion of bookings which are revenue-generating in 2020 YTD as compared to 2019 YTD is not supported by a proportional increase in capture of non-refundable cancellations. This is healthy, as dependency on such a mechanism is based on luck / is non-predicatable.<br>
That said, the "Groups" segment has respectively 19% then 17% of revenue-generating bookings being this type of cancelled, non-refundable one. Aim to minimise.

#### Lead Time

Lead time := time between making of reservation and actual check-in date.<br>
Min = 0, Max = 709 ... assume means days

# df_2019_vs_2020[df_2019_vs_2020.lead_time > 0].lead_time.min()
df_2019_vs_2020[df_2019_vs_2020.lead_time > 0].lead_time.max()

# pd.options.display.max_rows = 450
df_2019_vs_2020_all.groupby("lead_time", as_index = False).size()
# Take this to mean no outliers, so lead_time.mean() is valid

# pd.options.display.max_rows = 50

df_2020_segments_lead = df_2019_vs_2020_all.groupby(["arrival_date_year", "is_canceled", "market_segment"])["lead_time"].mean().reset_index().rename(columns = {"lead_time" : "mean_lead_time"})

df_2020_segments_lead_pivot = df_2020_segments_lead.pivot(index= ["market_segment", "is_canceled"], columns = "arrival_date_year", values = "mean_lead_time")

df_2020_segments_lead_pivot["delta"] = df_2020_segments_lead_pivot[2020] - df_2020_segments_lead_pivot[2019]
df_2020_segments_lead_pivot["pc_delta"] = df_2020_segments_lead_pivot["delta"] / df_2020_segments_lead_pivot[2019]

df_2020_segments_lead_pivot.columns.name = "mean_lead_time"
df_2020_segments_lead_pivot

For those same important market segments, mean lead time has increased whether for cancellations or completed bookings.<br>
Longer lead time for completed bookings --> customers are more confident in their purchasing decision, so this bodes well - they are booking in advance.<br>

However, there is also a rising trend of customers that have booked far in advance (long lead time) and cancelled too.<br>
Indeed, not only has "Online TA" increased in actual number of cancellations and decreased in % completed, but we can now see that their cancelled bookings have longer lead-times on average<br>
This overall implies volatility in this market segment.<br>
Perhaps consider early booking incentives for non-refundable rates